In [32]:
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras import layers, applications, optimizers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, GlobalMaxPooling2D, BatchNormalization, ZeroPadding2D, Input, Add
from keras.preprocessing.image import ImageDataGenerator, load_img

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random
import os

In [33]:
image_size = 224
batch_size = 32

save_model_filename = 'cassava_resnet.h5'

In [34]:
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')

In [35]:
train_path = '../input/cassava-leaf-disease-classification/train_images'

In [36]:
def image_path(image):
    return os.path.join(train_path,image)

train['image_id'] = train['image_id'].apply(image_path)

In [37]:
train['label'] = train['label'].astype('str')

In [38]:
image_generator = ImageDataGenerator(rotation_range=15, horizontal_flip=True, vertical_flip=True,
                                     shear_range=0.2, zoom_range=0.2, fill_mode='nearest', 
                                     validation_split=0.2, rescale = 1./255,
                                     preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

In [39]:
train_generator = image_generator.flow_from_dataframe(dataframe=train, directory=None, x_col='image_id', y_col='label',
                                                      subset='training', color_mode='rgb', batch_size=batch_size, seed=42,
                                                      shuffle=True, class_mode='categorical', target_size=(image_size,image_size))

In [40]:
validation_generator = image_generator.flow_from_dataframe(dataframe=train, directory=None, x_col='image_id', y_col='label',
                                                   subset='validation', color_mode='rgb', batch_size=batch_size, seed=42,
                                                   shuffle=False, class_mode='categorical', target_size=(image_size,image_size))

In [41]:
def build_resnet_block(input_layer, num_cnn=3, channel=64, block_num=1):

    x = input_layer
    
    for cnn_num in range(num_cnn):
        shortcut = x
        x = keras.layers.Conv2D(filters=channel, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal',
            padding='same', name=f'block{block_num}_conv{cnn_num}')(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Conv2D( filters=channel, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal',
            padding='same', name=f'block{block_num}_1_conv{cnn_num}')(x)

        shortcut_channel = shortcut.shape.as_list()[-1]
        if shortcut_channel != channel:
            shortcut = Conv2D(channel, kernel_size=(1,1), strides=(1,1), padding='same')(shortcut)  
        # skip connection
        x = keras.layers.Add()([x, shortcut])

    return x

In [42]:
def build_resnet(input_shape=(image_size, image_size, 3),
              num_cnn_list=[3, 4, 6, 3], channel_list=[64, 128, 256, 512], num_classes=5):
        
    input_layer = keras.layers.Input(shape=input_shape)
    
    output = input_layer
    
    output = keras.layers.Conv2D(filters=64, kernel_size=(7,7), strides=2, padding = 'same')(output)
    output = keras.layers.BatchNormalization()(output)
    output = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2)(output)
    
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        output = build_resnet_block(output, num_cnn=num_cnn, channel=channel, block_num=i)
        
    output = keras.layers.AveragePooling2D(padding = 'same')(output)
    output = keras.layers.Flatten(name='flatten')(output)
    output = keras.layers.Dense(1000, activation='relu', name='fc1')(output)    
    output = keras.layers.Dense(num_classes, activation='softmax', name='softmax')(output)
    
    model = keras.Model(
        inputs=input_layer,
        outputs=output
    )
    
    return model

In [43]:
model = build_resnet()

In [44]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9),
    metrics=['accuracy']
)

In [45]:
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1)

In [ ]:
epoch = 30

history = model.fit(
    train_generator,
    epochs=epoch,
    validation_data=validation_generator,
    verbose=1,
    callbacks=[reduce_lr]
)

In [ ]:
test_path = '../input/cassava-leaf-disease-classification/test_images'

In [ ]:
test_filenames = list(os.listdir(test_path))
print(test_filenames)

In [ ]:
x_test = []

for filename in test_filenames:
    image = load_img(os.path.join(test_path, filename), color_mode='rgb', target_size=(image_size,image_size))
    image = np.array(image)
    x_test.append(image)
    
x_test = np.array(x_test)
x_test = x_test.astype('float32')
x_test /= 255.0

In [ ]:
y_test = model.predict(x_test)

In [ ]:
y_test = np.argmax(y_test, axis=1)

In [ ]:
submission = pd.DataFrame((zip(test_filenames, y_test)), columns=['image_id', 'label'])

submission.to_csv('submission.csv', index=False)